In [ ]:
# install packages
# remotes::install_github("jokergoo/circlize@9b21578")
# remotes::install_github("jokergoo/ComplexHeatmap@7d95ca5")
# remotes::install_github("immunogenomics/presto@31dc97f")
remotes::install_github("jinworks/CellChat@88c2e13")
BiocManager::install("GenomeInfoDbData")

In [ ]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(zellkonverter)
    library(scater)
    library(scran)
    library(scuttle)
    library(SingleCellExperiment)
    library(CellChat)
})

In [ ]:
sce = readH5AD("tmp/adata_gated.h5ad")

In [ ]:
sce$day <- stringr::str_extract(sce$batch, "\\d+")
sce$day <- factor(sce$day, levels = c("6", "8", "30", "90"))

In [ ]:
run_cellchat <- function(sce_tmp, name) {
    

    data.input = assay(sce_tmp, "X") # X are the log norm counts here, see part 1
    meta = data.frame(labels = sce_tmp$Subtype_gate,
                    samples = sce_tmp$batch,
                    row.names = colnames(sce_tmp))
    spatial.locs = reducedDim(sce_tmp, 'X_spatial') |> as.data.frame()
    scale.factors = list(spot.diameter = 5, spot = 5)
    spatial.factors = data.frame(ratio = 1, tol = 5)

    cellchat <-
        createCellChat(
            object = data.input,
            meta = meta,
            group.by = "labels",
            datatype = "spatial",
            coordinates = spatial.locs,
            spatial.factors = spatial.factors
        )


    CellChatDB <- CellChatDB.mouse # use CellChatDB.human if running on human data

    # use a subset of CellChatDB for cell-cell communication analysis
    # CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling") # use Secreted Signaling
    # use all CellChatDB for cell-cell communication analysis
    CellChatDB.use <- CellChatDB # simply use the default CellChatDB

    # set the used database in the object
    cellchat@DB <- CellChatDB.use

    # subset the expression data of signaling genes for saving computation cost
    cellchat <- subsetData(cellchat) # This step is necessary even if using the whole database

    # future::plan("multisession", workers = 8) # do parallel
    cellchat <- identifyOverExpressedGenes(cellchat)
    cellchat <- identifyOverExpressedInteractions(cellchat)

    # Typically, contact.range = 10, which is a typical human cell size
    cellchat <- computeCommunProb(cellchat,
        type = "truncatedMean", trim = 0.1,
        distance.use = TRUE, interaction.range = 100,
        scale.distance = 1,
        contact.dependent = TRUE, contact.range = 10
    )
    # Filter out the cell-cell communication if there are only few number of cells in certain cell groups
    cellchat <- filterCommunication(cellchat, min.cells = 10)

    cellchat <- computeCommunProbPathway(cellchat)
    cellchat <- aggregateNet(cellchat)

    saveRDS(cellchat, file = paste0("tmp/cellchat_",name,".rds"))
}

In [ ]:
cellchat_day <- function(sce, day) {
    sce_tmp = sce[,sce$day == day]

    # This is absolutely key. Otherwise Cellchat does not WORK!
    sce_tmp$batch <- droplevels(sce_tmp$batch)
    
    run_cellchat(sce_tmp, day)
}

In [ ]:
cellchat_day(sce, 6)
cellchat_day(sce, 8)
cellchat_day(sce, 30)
cellchat_day(sce, 90)

In [ ]:
run_cellchat(sce, "full")

In [ ]:
# data.input = assay(sce, "X") # X are the log norm counts here, see part 1
# meta = data.frame(labels = sce$Subtype_gate,
#                   slices = sce$batch,
#                   row.names = colnames(sce))
# spatial.locs = reducedDim(sce, 'X_spatial') |> as.data.frame()
# scale.factors = list(spot.diameter = 5, spot = 5)
# spatial.factors = data.frame(ratio = 1, tol = 5)

In [ ]:
# cellchat <-
#   createCellChat(
#     object = data.input,
#     meta = meta,
#     group.by = "labels",
#     datatype = "spatial",
#     coordinates = spatial.locs,
#     spatial.factors = spatial.factors
#   )

In [ ]:
# cellchat

In [ ]:
# CellChatDB <- CellChatDB.mouse # use CellChatDB.human if running on human data

# # use a subset of CellChatDB for cell-cell communication analysis
# # CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling") # use Secreted Signaling
# # use all CellChatDB for cell-cell communication analysis
# CellChatDB.use <- CellChatDB # simply use the default CellChatDB

# # set the used database in the object
# cellchat@DB <- CellChatDB.use

In [ ]:
# # subset the expression data of signaling genes for saving computation cost
# cellchat <- subsetData(cellchat) # This step is necessary even if using the whole database


In [ ]:
# # future::plan("multisession", workers = 8) # do parallel
# cellchat <- identifyOverExpressedGenes(cellchat)
# cellchat <- identifyOverExpressedInteractions(cellchat)

This part takes a long time to run!
(~3h)

In [ ]:
# # Typically, contact.range = 10, which is a typical human cell size
# cellchat <- computeCommunProb(cellchat,
#   type = "truncatedMean", trim = 0.1,
#   distance.use = TRUE,, interaction.range = 100,
#   scale.distance = 1,
#   contact.dependent = TRUE, contact.range = 10
# )
# # Filter out the cell-cell communication if there are only few number of cells in certain cell groups
# cellchat <- filterCommunication(cellchat, min.cells = 10)


In [ ]:
# cellchat <- computeCommunProbPathway(cellchat)
# cellchat <- aggregateNet(cellchat)

In [ ]:
# saveRDS(cellchat, file = "tmp/cellchat.rds")